## Introducción al Sistema RAG para Discursos del Presidente Javier Milei

Este cuaderno de Google Colab presenta un sistema de **Generación Aumentada por Recuperación (RAG)** diseñado específicamente para analizar y consultar los discursos del presidente Javier Milei. Nuestro objetivo es permitir a usuarios interesados en las políticas de Milei consultar sus posturas, propuestas y argumentos a lo largo del tiempo, basándose en un corpus en constante actualización de sus discursos, comunicados y diversas intervenciones públicas.

### ¿Cómo funciona este sistema?

1.  **Recuperación Inteligente:** Utilizamos **ChromaDB** para realizar búsquedas semánticas. Esto significa que el sistema entiende el "significado" de tu pregunta, no solo las palabras clave, para encontrar los fragmentos más relevantes de los discursos.
2.  **Generación de Respuestas:** La generación de respuestas se potencia con el modelo **Gemini** de Google. Gemini toma los fragmentos relevantes encontrados y los utiliza para formular respuestas coherentes, precisas y contextualizadas.

Este enfoque garantiza que las respuestas no solo sean informativas, sino que también estén directamente respaldadas por las fuentes originales de los discursos, minimizando las "alucinaciones" y proporcionando una base sólida para el análisis de las políticas presidenciales.

## Configuración del Entorno

Primero instalamos las herramientas necesarias. LangChain es nuestra "caja de herramientas" principal para RAG.

In [ ]:
# Instalamos las librerías necesarias para nuestro sistema RAG
# LangChain: la librería más popular para construir sistemas RAG de forma simple
# ChromaDB: nuestra base de datos vectorial para guardar los documentos
# Google GenerativeAI: para conectar con Gemini (solo para generación final)
# sentence-transformers: para embeddings locales multilenguaje
!pip install langchain langchain-google-genai langchain-chroma chromadb sentence-transformers -q

print("Todas las librerías instaladas correctamente")
print("IMPORTANTE: Solo se usará API de Gemini para generación final de respuestas")

Todas las librerías instaladas correctamente
IMPORTANTE: Solo se usará API de Gemini para generación final de respuestas


In [ ]:
# Importamos todas las herramientas que vamos a necesitar
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain.chains import RetrievalQA
from langchain.schema import Document
from langchain.prompts import PromptTemplate
from chromadb.utils import embedding_functions

# Configuración para mostrar mejor los resultados
import warnings
warnings.filterwarnings('ignore')

print("Librerías cargadas exitosamente")
print("Usando embeddings locales para reducir uso de API")

Librerías cargadas exitosamente
Usando embeddings locales para reducir uso de API


## Configuración de la API de Gemini

Necesitamos configurar nuestra conexión con Gemini de Google. Este será el "cerebro" que generará las respuestas finales.

In [ ]:
# Detectamos si estamos en Google Colab o en un entorno local
try:
    import google.colab
    from google.colab import userdata
    IN_COLAB = True
    print("🔍 Entorno detectado: Google Colab")
except ImportError:
    IN_COLAB = False
    print("🔍 Entorno detectado: Local")

# Obtenemos la clave API según el entorno
if IN_COLAB:
    # En Colab: usar los secretos de Colab (más seguro)
    try:
        GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
        print("✅ Clave API cargada desde secretos de Colab")
    except Exception as e:
        print("❌ No se encontró GOOGLE_API_KEY en los secretos de Colab")
        print("   Ve a la barra lateral izquierda > 🔑 Secretos > Agregar GOOGLE_API_KEY")
        GOOGLE_API_KEY = input("Pega tu clave API de Google aquí: ")
else:
    # En local: usar variable de entorno
    GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
    if not GOOGLE_API_KEY:
        print("❌ No se encontró GOOGLE_API_KEY en las variables de entorno")
        print("   Opción 1: Agrega GOOGLE_API_KEY a tu archivo .env")
        print("   Opción 2: Ejecuta: export GOOGLE_API_KEY=tu_clave_aqui")
        GOOGLE_API_KEY = input("Pega tu clave API de Google aquí: ")
    else:
        print("✅ Clave API cargada desde variables de entorno")

# Configuramos la variable de entorno para que LangChain la use
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
print("🚀 Configuración de Gemini completada")

🔍 Entorno detectado: Google Colab
✅ Clave API cargada desde secretos de Colab
🚀 Configuración de Gemini completada


## Web Scraping de Discursos de Casa Rosada

Este cuaderno contiene un script de Python diseñado para realizar web scraping en el sitio web de la Casa Rosada y extraer los discursos públicos del Presidente. El objetivo principal es recolectar información relevante de estos discursos para su posterior análisis.

### Funcionalidades Clave:

1.  **Extracción de Lista de Discursos**: Navega a la sección de "Discursos" de la Casa Rosada y extrae los enlaces y títulos de los discursos más recientes.
2.  **Extracción de Contenido Individual**: Para cada discurso identificado, el script accede a su página individual para extraer el título, la fecha de publicación, el contenido completo (crudo) y realiza una limpieza del texto para eliminar ruido y formatearlo adecuadamente.
3.  **Procesamiento y Estadísticas**: Calcula estadísticas básicas para cada discurso, como la longitud del texto (caracteres y palabras) y el número de párrafos.
4.  **Guardado de Resultados**: Al finalizar el proceso, los datos recolectados son guardados en dos formatos: un archivo CSV y un archivo JSON, lo que facilita su uso en diferentes herramientas de análisis.
5.  **Resumen y Estadísticas Finales**: Muestra un resumen detallado de cada discurso extraído y proporciona estadísticas globales del conjunto de datos.

Este script es una herramienta útil para investigadores, periodistas o cualquier persona interesada en el análisis de los discursos presidenciales argentinos.

In [ ]:
# @title
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime
import time
import os

class ScraperDiscursosCasaRosada:
    def __init__(self):
        self.base_url = "https://www.casarosada.gob.ar/informacion/discursos"
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        self.discursos = []

    def extraer_lista_discursos(self):
        """
        Extrae la lista de los últimos discursos de la página principal
        """
        try:
            print(f"🌐 Conectando a: {self.base_url}")
            response = requests.get(self.base_url, headers=self.headers, timeout=10)
            response.raise_for_status()

            soup = BeautifulSoup(response.content, 'html.parser')
            print("✅ Página cargada correctamente")

            # Estrategia más simple: buscar todos los enlaces y filtrar inteligentemente
            todos_los_enlaces = soup.find_all('a', href=True)
            print(f"🔍 Encontrados {len(todos_los_enlaces)} enlaces en total")

            discursos_links = []

            for enlace in todos_los_enlaces:
                texto = enlace.get_text(strip=True)
                href = enlace['href']

                # Filtrar enlaces que NO queremos
                if (not texto or
                    len(texto) < 10 or  # Reducido de 15 a 10 para más flexibilidad
                    texto.lower() in ['discursos', 'inicio', 'volver', 'volver al inicio', 'ir al contenido'] or
                    '#' in href or  # Enlaces internos
                    'javascript:' in href.lower()):
                    continue

                # Construir URL completa
                if href.startswith('/'):
                    url_completa = f"https://www.casarosada.gob.ar{href}"
                elif href.startswith('http'):
                    url_completa = href
                else:
                    url_completa = f"https://www.casarosada.gob.ar/{href}"

                # Incluir enlaces que parezcan ser discursos individuales
                # Ser más permisivo con los criterios
                es_posible_discurso = (
                    # Criterio 1: Contiene palabras clave
                    any(palabra in texto.lower() for palabra in [
                        'discurso', 'milei', 'presidente', 'palabras', 'alocución',
                        'mensaje', 'javier', 'conferencia', 'declaración', 'ministro',
                        'argentina', 'nación', 'gobierno', 'política', 'economía'
                    ]) or
                    # Criterio 2: Tiene formato de fecha + título
                    (re.search(r'\d{1,2}\s+de\s+\w+', texto) and len(texto) > 20) or  # Reducido de 30 a 20
                    # Criterio 3: URL contiene patrones de discursos
                    any(patron in url_completa.lower() for patron in [
                        '/discursos/', '/discurso', 'noticia', 'article', 'informacion'
                    ]) or
                    # Criterio 4: Texto suficientemente largo y parece contenido
                    (len(texto) > 25 and any(caracter in texto for caracter in [':', '-', '–']))
                )

                if es_posible_discurso:
                    discursos_links.append({
                        'titulo': texto,
                        'url': url_completa,
                        'fecha_extraccion': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    })

            # Eliminar duplicados por URL
            discursos_unicos = []
            urls_vistas = set()

            for discurso in discursos_links:
                # Normalizar URL para comparación
                url_normalizada = discurso['url'].split('?')[0]  # Remover parámetros
                if url_normalizada not in urls_vistas:
                    discursos_unicos.append(discurso)
                    urls_vistas.add(url_normalizada)

            print(f"📋 Filtrados {len(discursos_unicos)} discursos potenciales")

            # Ordenar por relevancia (los que tienen fechas primero)
            def relevancia(titulo):
                puntaje = 0
                if re.search(r'\d{1,2}\s+de\s+\w+\s+de\s+\d{4}', titulo):
                    puntaje += 3
                if 'discurso' in titulo.lower():
                    puntaje += 2
                if 'milei' in titulo.lower():
                    puntaje += 2
                if 'presidente' in titulo.lower():
                    puntaje += 1
                if len(titulo) > 50:  # Títulos más largos suelen ser más específicos
                    puntaje += 1
                return puntaje

            discursos_unicos.sort(key=lambda x: relevancia(x['titulo']), reverse=True)

            # Devolver los 10 más relevantes
            return discursos_unicos[:10]

        except Exception as e:
            print(f"❌ Error al extraer lista de discursos: {e}")
            return []

    def extraer_contenido_discurso(self, url_discurso):
        """
        Extrae el contenido completo de un discurso individual
        """
        try:
            print(f"   📖 Extrayendo contenido de: {url_discurso}")
            response = requests.get(url_discurso, headers=self.headers, timeout=10)
            response.raise_for_status()

            soup = BeautifulSoup(response.content, 'html.parser')

            # Extraer título - buscar en diferentes ubicaciones
            titulo_selectores = [
                'h1',
                '.titulo',
                '.title',
                'header h1',
                'main h1',
                'article h1',
                '.entry-title',
                '.post-title',
                '.noticia-titulo',
                '.page-title',
                '.node-title'
            ]

            titulo = None
            for selector in titulo_selectores:
                titulo_elem = soup.select_one(selector)
                if titulo_elem:
                    titulo = titulo_elem
                    break

            titulo_texto = titulo.get_text(strip=True) if titulo else "Sin título"

            # Estrategias para encontrar el contenido principal
            selectores_contenido = [
                'article .contenido',
                'article .content',
                'article .texto',
                '.nota-contenido',
                '.entry-content',
                '.post-content',
                '.article-content',
                '.field-body',
                '.field-content',
                'main article',
                'article',
                '.contenido',
                '.content',
                '.texto',
                'main',
                '.node-content'
            ]

            contenido = None
            for selector in selectores_contenido:
                contenido_elem = soup.select_one(selector)
                if contenido_elem:
                    contenido = contenido_elem
                    break

            # Si no encontramos con selectores específicos, buscar el área principal de texto
            if not contenido:
                # Buscar el elemento que contiene la mayor cantidad de texto
                elementos_texto = soup.find_all(['div', 'section', 'article'])
                if elementos_texto:
                    # Filtrar elementos con suficiente texto
                    elementos_con_texto = [elem for elem in elementos_texto if len(elem.get_text(strip=True)) > 200]
                    if elementos_con_texto:
                        contenido = max(elementos_con_texto, key=lambda x: len(x.get_text()))

            texto_completo = contenido.get_text(separator='\n', strip=True) if contenido else ""

            # Extraer fecha - buscar en diferentes formatos
            fecha_selectores = [
                'time',
                '.fecha',
                '.date',
                '.fecha-publicacion',
                '.entry-date',
                '.post-date',
                'span.fecha',
                '.fecha-noticia',
                '.created',
                '.publish-date'
            ]

            fecha_element = None
            for selector in fecha_selectores:
                fecha_elem = soup.select_one(selector)
                if fecha_elem:
                    fecha_element = fecha_elem
                    break

            # Si no encontramos fecha con selectores, buscar en el texto
            if not fecha_element:
                # Buscar patrones de fecha en el contenido
                fecha_pattern = re.search(r'\d{1,2}\s+de\s+\w+\s+de\s+\d{4}', texto_completo)
                if fecha_pattern:
                    fecha = fecha_pattern.group()
                else:
                    fecha = "Fecha no disponible"
            else:
                fecha = fecha_element.get_text(strip=True)

            return {
                'titulo': titulo_texto,
                'contenido_crudo': texto_completo,
                'fecha_publicacion': fecha,
                'url': url_discurso
            }

        except Exception as e:
            print(f"   ❌ Error al extraer discurso: {e}")
            return None

    def limpiar_texto(self, texto):
        """
        Limpia y preprocesa el texto extraído
        """
        if not texto:
            return ""

        # 1. Eliminar espacios múltiples y normalizar saltos de línea
        texto = re.sub(r'\n+', '\n', texto)
        texto = re.sub(r' +', ' ', texto)

        # 2. Eliminar caracteres especiales no deseados (mantener puntuación básica y acentos)
        texto = re.sub(r'[^\w\s.,;:!?¿¡()\-áéíóúÁÉÍÓÚñÑ@]', '', texto)

        # 3. Normalizar saltos de línea y espacios
        lineas = [linea.strip() for linea in texto.split('\n') if linea.strip()]
        texto_limpio = '\n'.join(lineas)

        # 4. Eliminar líneas muy cortas que puedan ser ruido (pero mantener párrafos válidos)
        lineas_filtradas = []
        for linea in texto_limpio.split('\n'):
            linea_limpia = linea.strip()
            if (len(linea_limpia) > 10 or  # Reducido de 15 a 10
                any(caracter in linea_limpia for caracter in ['.', '!', '?', ':', ';']) and len(linea_limpia) > 5):  # Reducido de 8 a 5
                lineas_filtradas.append(linea_limpia)

        return '\n'.join(lineas_filtradas)

    def procesar_discursos(self):
        """
        Proceso completo de ingesta: extracción y limpieza de los últimos 10 discursos
        """
        print("🔍 Extrayendo lista de discursos...")
        lista_discursos = self.extraer_lista_discursos()

        if not lista_discursos:
            print("❌ No se pudieron extraer los discursos desde la página")
            return []

        print(f"📝 Procesando {len(lista_discursos)} discursos...")

        discursos_exitosos = 0
        for i, discurso_info in enumerate(lista_discursos, 1):
            print(f"  [{i}/{len(lista_discursos)}] 📄 Procesando: {discurso_info['titulo'][:80]}...")

            contenido = self.extraer_contenido_discurso(discurso_info['url'])
            if contenido and len(contenido['contenido_crudo']) > 100:  # Solo si tiene contenido válido
                # Aplicar limpieza
                contenido_limpio = self.limpiar_texto(contenido['contenido_crudo'])

                # Solo incluir si el contenido limpio tiene suficiente texto
                if len(contenido_limpio) > 200:
                    discurso_procesado = {
                        **discurso_info,
                        **contenido,
                        'contenido_limpio': contenido_limpio,
                        'longitud_texto': len(contenido_limpio),
                        'num_parrafos': contenido_limpio.count('\n') + 1,
                        'palabras_aprox': len(contenido_limpio.split())
                    }

                    self.discursos.append(discurso_procesado)
                    discursos_exitosos += 1
                    print(f"     ✅ Extraído: {len(contenido_limpio)} caracteres, {discurso_procesado['palabras_aprox']} palabras aprox.")
                else:
                    print(f"     ⚠️  Descartado: contenido muy corto después de limpieza")
            else:
                print(f"     ❌ Falló la extracción o contenido insuficiente")

            time.sleep(1)  # Respetuoso con el servidor

            # Si ya tenemos 10 discursos exitosos, podemos parar
            if discursos_exitosos >= 10:
                print(f"🎯 Ya se alcanzaron {discursos_exitosos} discursos exitosos, continuando con los restantes...")

        print(f"📊 Resumen: {discursos_exitosos}/{len(lista_discursos)} discursos extraídos exitosamente")
        return self.discursos

    def guardar_resultados(self, formato='csv'):
        """
        Guarda los resultados en diferentes formatos
        """
        if not self.discursos:
            print("No hay datos para guardar")
            return

        df = pd.DataFrame(self.discursos)

        # Crear directorio si no existe
        os.makedirs('resultados_discursos', exist_ok=True)

        timestamp = datetime.now().strftime('%Y%m%d_%H%M')

        if formato == 'csv':
            filename = f"resultados_discursos/discursos_casarosada_10_{timestamp}.csv"
            df.to_csv(filename, index=False, encoding='utf-8-sig')
        elif formato == 'json':
            filename = f"resultados_discursos/discursos_casarosada_10_{timestamp}.json"
            df.to_json(filename, orient='records', force_ascii=False, indent=2)

        print(f"💾 Resultados guardados en: {filename}")
        return filename

    def mostrar_resumen(self):
        """Muestra un resumen de los discursos extraídos"""
        if not self.discursos:
            print("No hay discursos para mostrar")
            return

        print("\n" + "="*80)
        print("📊 RESUMEN DE 10 DISCURSOS EXTRAÍDOS")
        print("="*80)

        for i, discurso in enumerate(self.discursos, 1):
            print(f"\n🎯 DISCURSO {i}:")
            print(f"   📖 Título: {discurso['titulo']}")
            print(f"   📅 Fecha publicación: {discurso['fecha_publicacion']}")
            print(f"   📊 Estadísticas:")
            print(f"      - Caracteres: {discurso['longitud_texto']:,}")
            print(f"      - Palabras aprox.: {discurso['palabras_aprox']:,}")
            print(f"      - Párrafos: {discurso['num_parrafos']}")
            print(f"   🔗 URL: {discurso['url'][:100]}...")

# EJECUCIÓN PRINCIPAL
print("🚀 INICIANDO SCRAPING DE 10 DISCURSOS DE LA CASA ROSADA")
print("="*60)

scraper = ScraperDiscursosCasaRosada()

# Ejecutar el scraping
discursos_procesados = scraper.procesar_discursos()

if discursos_procesados:
    print(f"\n🎉 PROCESAMIENTO COMPLETADO!")
    print(f"✅ Se extrajeron y procesaron {len(discursos_procesados)} discursos")

    # Mostrar resumen
    scraper.mostrar_resumen()

    # Guardar resultados
    print(f"\n💾 GUARDANDO RESULTADOS...")
    archivo_csv = scraper.guardar_resultados('csv')
    archivo_json = scraper.guardar_resultados('json')

    print(f"\n📁 Archivos generados:")
    print(f"   - CSV: {archivo_csv}")
    print(f"   - JSON: {archivo_json}")

    # Estadísticas finales
    total_caracteres = sum(d['longitud_texto'] for d in discursos_procesados)
    total_palabras = sum(d['palabras_aprox'] for d in discursos_procesados)

    print(f"\n📈 ESTADÍSTICAS FINALES:")
    print(f"   📚 Total de discursos: {len(discursos_procesados)}")
    print(f"   🔤 Total de caracteres: {total_caracteres:,}")
    print(f"   🗣️  Total de palabras aprox.: {total_palabras:,}")
    print(f"   📅 Rango temporal: {min(d['fecha_publicacion'] for d in discursos_procesados) if discursos_procesados else 'N/A'} a {max(d['fecha_publicacion'] for d in discursos_procesados) if discursos_procesados else 'N/A'}")

else:
    print("❌ No se pudieron extraer discursos")

🚀 INICIANDO SCRAPING DE 10 DISCURSOS DE LA CASA ROSADA
🔍 Extrayendo lista de discursos...
🌐 Conectando a: https://www.casarosada.gob.ar/informacion/discursos
✅ Página cargada correctamente
🔍 Encontrados 112 enlaces en total
📋 Filtrados 50 discursos potenciales
📝 Procesando 10 discursos...
  [1/10] 📄 Procesando: Jueves 06 de Noviembre de 2025Discurso del Presidente Javier Milei en el America...
   📖 Extrayendo contenido de: https://www.casarosada.gob.ar/informacion/discursos/51109-discurso-del-presidente-javier-milei-en-el-america-business-forum-de-miami
     ✅ Extraído: 23867 caracteres, 4033 palabras aprox.
  [2/10] 📄 Procesando: Jueves 06 de Noviembre de 2025Discurso del Presidente Javier Milei en la Cena de...
   📖 Extrayendo contenido de: https://www.casarosada.gob.ar/informacion/discursos/51111-discurso-del-presidente-javier-milei-en-la-cena-de-gala-del-cpac-en-mar-a-lago-miami
     ✅ Extraído: 8256 caracteres, 1378 palabras aprox.
  [3/10] 📄 Procesando: Viernes 24 de Octubre de 2

## Dividir los Documentos en Fragmentos

Los documentos largos necesitan dividirse en pedazos más pequeños para que el sistema pueda encontrar información específica. Es como hacer un índice detallado de un libro.

In [ ]:
# El "Text Splitter" es como un bibliotecario que divide documentos grandes
# en secciones manejables, manteniendo el contexto

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,        # Cada fragmento tendrá máximo 500 caracteres
    chunk_overlap=50,      # 50 caracteres se superponen entre fragmentos para mantener contexto
    separators=["\n\n", "\n", ".", " "]  # Divide preferentemente por párrafos, luego oraciones
)

# Convertimos nuestros documentos al formato que entiende LangChain
documentos_langchain = []

# Ahora usamos los discursos procesados del web scraping
for doc in discursos_procesados:
    # Cada documento se convierte en un objeto "Document" con contenido y metadata
    documento = Document(
        page_content=doc["contenido_limpio"],
        metadata={
            "titulo": doc["titulo"],
            "fecha_publicacion": doc["fecha_publicacion"],
            "url": doc["url"]
        }
    )
    documentos_langchain.append(documento)

# Dividimos todos los documentos en fragmentos más pequeños
fragmentos = text_splitter.split_documents(documentos_langchain)

print(f"📝 Discursos procesados: {len(discursos_procesados)}")
print(f"🔪 Fragmentos creados: {len(fragmentos)}")
print(f"\n📋 Ejemplo de fragmento:")
print(f"Título: {fragmentos[0].metadata['titulo']}")
print(f"Fecha: {fragmentos[0].metadata['fecha_publicacion']}")
print(f"Contenido: {fragmentos[0].page_content[:200]}...")

📝 Discursos procesados: 10
🔪 Fragmentos creados: 502

📋 Ejemplo de fragmento:
Título: Casa RosadaPresidencia de la Nación
Fecha: Jueves 06 de noviembre de 2025
Contenido: Discurso del Presidente Javier Milei en el America Business Forum de Miami
Hola a todos. No, gracias a ustedes por favor, gracias por recibirme con tanto cariño. Muchas gracias. Absolutamente que vamo...


In [ ]:
print(f"Título: {fragmentos[2].metadata['titulo']}")
print(f"Contenido: {fragmentos[2].page_content[:200]}...")

Título: Casa RosadaPresidencia de la Nación
Contenido: . También me alegra poder compartir escenario con uno de nuestros más ilustres deportistas y orgullo de todos los argentinos, Lionel Messi, un hombre que llevó el talento argentino a lo más alto del m...


## Crear la Base de Conocimiento Vectorial

Ahora convertimos nuestros fragmentos de texto en "vectores" (listas de números) que el sistema puede comparar y buscar de forma inteligente.

In [ ]:
# Los "embeddings" convierten texto en vectores numéricos que representan el significado
# Usamos un modelo local multilenguaje que funciona excelente con español técnico
embeddings = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="intfloat/multilingual-e5-large"  # Modelo multilenguaje optimizado para español
)

print("Modelo de embeddings local configurado (multilingual-e5-large)")
print("Ventaja: No consume cuota de API, solo procesamiento local")

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

Modelo de embeddings local configurado (multilingual-e5-large)
Ventaja: No consume cuota de API, solo procesamiento local


In [ ]:
!pip install langchain_community -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.37.0 requires opentelemetry-exporter-otlp-proto-common==1.37.0, but you have opentelemetry-exporter-otlp-proto-common 1.38.0 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.37.0 requires opentelemetry-proto==1.37.0, but you have opentelemetry-proto 1.38.0 which is incompatible.


In [ ]:
# ChromaDB será nuestra "biblioteca inteligente" donde guardamos los vectores
# Es como un bibliotecario que puede encontrar libros por su tema, no solo por título
from langchain_community.embeddings import SentenceTransformerEmbeddings

embeddings = SentenceTransformerEmbeddings(model_name="intfloat/multilingual-e5-large")

vectorstore = Chroma.from_documents(
    documents=fragmentos,           # Los fragmentos de nuestros documentos
    embedding=embeddings,           # El modelo que convierte texto en vectores
    collection_name="documentos_empresa",  # Nombre de nuestra colección
    persist_directory="./chroma_db"  # Donde se guardan los datos (opcional)
)

print(f"Base de conocimiento vectorial creada con {len(fragmentos)} fragmentos")
print("El sistema ya puede buscar información por significado, no solo por palabras exactas")
print("Los embeddings se procesan localmente sin consumir cuota de Gemini")

Base de conocimiento vectorial creada con 502 fragmentos
El sistema ya puede buscar información por significado, no solo por palabras exactas
Los embeddings se procesan localmente sin consumir cuota de Gemini


In [ ]:
# Configuramos el modelo Gemini que generará las respuestas finales
# NOTA: Solo este componente consume cuota de API, los embeddings son locales
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",    # Modelo rápido y eficiente de Gemini
    temperature=0.1,             # Baja creatividad = respuestas más precisas y consistentes
    google_api_key=GOOGLE_API_KEY
)

print("Modelo Gemini configurado")
print("   Modelo: gemini-2.5-flash (rápido y preciso)")
print("   Temperatura: 0.1 (respuestas consistentes y factuales)")
print("   IMPORTANTE: Solo la generación final usa API de Gemini")

Modelo Gemini configurado
   Modelo: gemini-2.5-flash (rápido y preciso)
   Temperatura: 0.1 (respuestas consistentes y factuales)
   IMPORTANTE: Solo la generación final usa API de Gemini


## Template de Respuesta

Definimos cómo queremos que Gemini estructure sus respuestas. Es como darle instrucciones específicas sobre su rol y forma de responder.

In [ ]:
# El prompt template es como las instrucciones que le damos a un asistente
# Le decimos exactamente como debe comportarse y que formato usar

# Template mejorado con mas estructura
template_mejorado = """
    Sos un asistente especializado en discursos y políticas del presidente Javier Milei.
    Tu misión es proporcionar información precisa y útil basada ÚNICAMENTE en
    los discursos oficiales disponibles.

    INSTRUCCIONES IMPORTANTES:
    1. Solo usá información que aparece EXPLÍCITAMENTE en los discursos
    2. Si no encontrás la información específica, indicá claramente "No se menciona en los discursos disponibles"
    3. Sé preciso con nombres, fechas, cifras y datos específicos
    4. Usá un tono amigable pero informado y profesional

    FORMATO DE RESPUESTA:
    - Comenzá con un resumen directo de lo encontrado
    - Proporcioná detalles específicos (nombres, fechas, eventos, cifras)
    - Si hay múltiples puntos, numerálos claramente
    - Mencioná en qué discurso y fecha se encuentra cada información
    - Si algo no está claro o no aparece en los documentos, decilo explícitamente

    DISCURSOS DISPONIBLES:
    {context}

    CONSULTA:
    {question}

    RESPUESTA ESTRUCTURADA:
    """

# Creamos el prompt personalizado usando nuestro template
prompt_mejorado = PromptTemplate(
    template=template_mejorado,
    input_variables=["context", "question"]
)

print("Template de respuesta configurado")
print("El asistente seguira instrucciones especificas para dar respuestas precisas")

Template de respuesta configurado
El asistente seguira instrucciones especificas para dar respuestas precisas


## Sistema RAG Completo Opcion FINAL


    retriever_complejo = vectorstore.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 8, "fetch_k": 25, "lambda_mult": 0.6}  # Más documentos para consultas complejas
    )



In [ ]:
# @title
# Función para manejar consultas complejas que requieren múltiples documentos
def consulta_compleja(pregunta):
    """
    Maneja consultas que pueden necesitar información de múltiples documentos
    """
    print(f"\n🧩 CONSULTA COMPLEJA")
    print(f"📋 Pregunta: {pregunta}")
    print("\n" + "="*80)

    # Configuramos para buscar más documentos
    retriever_complejo = vectorstore.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 8, "fetch_k": 25, "lambda_mult": 0.6}  # Más documentos para consultas complejas
    )

    # Sistema temporal con más contexto
    sistema_complejo = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever_complejo,
        chain_type_kwargs={"prompt": prompt_mejorado},
        return_source_documents=True
    )

    resultado = sistema_complejo({"query": pregunta})

    print(f"🤖 RESPUESTA INTEGRAL:")
    print(resultado["result"])

    print(f"\n" + "="*80)
    print(f"📊 ANÁLISIS DE FUENTES CONSULTADAS")
    print("="*80)

    # Análisis detallado de los documentos fuente
    documentos = resultado['source_documents']

    # Estadísticas generales
    documentos_unicos = list(set([doc.metadata.get('titulo', 'Título no disponible') for doc in documentos]))
    fechas_unicas = list(set([doc.metadata.get('fecha_publicacion', 'No disponible') for doc in documentos]))

    print(f"📈 ESTADÍSTICAS:")
    print(f"   • Total de fragmentos consultados: {len(documentos)}")
    print(f"   • Documentos únicos: {len(documentos_unicos)}")
    print(f"   • Rango temporal: {min(fechas_unicas) if fechas_unicas else 'N/A'} a {max(fechas_unicas) if fechas_unicas else 'N/A'}")

    print(f"\n📚 DOCUMENTOS CONSULTADOS (por relevancia):")
    print("-" * 80)

    # Agrupar por documento y mostrar fragmentos relevantes
    documentos_agrupados = {}
    for doc in documentos:
        titulo = doc.metadata.get('titulo', 'Título no disponible')
        if titulo not in documentos_agrupados:
            documentos_agrupados[titulo] = []
        documentos_agrupados[titulo].append(doc)

    for i, (titulo, fragments) in enumerate(documentos_agrupados.items(), 1):
        # Extraer información del documento
        fecha = fragments[0].metadata.get('fecha_publicacion', 'Fecha no disponible')
        url = fragments[0].metadata.get('url', 'URL no disponible')

        print(f"\n🔹 {i}. {titulo}")
        print(f"   📅 Fecha: {fecha}")
        print(f"   🔗 Fragmentos relevantes: {len(fragments)}")
        print(f"   🌐 URL: {url[:80]}..." if len(url) > 80 else f"   🌐 URL: {url}")

        # Mostrar los fragmentos más relevantes (primeros 2)
        print(f"   📖 Fragmentos destacados:")
        for j, fragment in enumerate(fragments[:2], 1):  # Mostrar solo los 2 primeros por documento
            contenido_limpio = fragment.page_content.replace('\n', ' ').strip()
            if len(contenido_limpio) > 150:
                contenido_limpio = contenido_limpio[:147] + "..."
            print(f"      {j}. {contenido_limpio}")

        if len(fragments) > 2:
            print(f"      ... y {len(fragments) - 2} fragmentos más")

    # Análisis de cobertura temática
    print(f"\n🎯 COBERTURA TEMÁTICA:")
    print("-" * 80)

    # Palabras clave de la pregunta
    palabras_clave = pregunta.lower().split()
    palabras_relevantes = [p for p in palabras_clave if len(p) > 3]  # Filtrar palabras muy cortas

    for palabra in palabras_relevantes[:5]:  # Analizar primeras 5 palabras clave
        menciones = 0
        for doc in documentos:
            if palabra in doc.page_content.lower():
                menciones += 1
        if menciones > 0:
            print(f"   • '{palabra}': mencionado en {menciones} fragmentos")

    # Calidad de las fuentes
    print(f"\n📋 RESUMEN EJECUTIVO DE FUENTES:")
    print("-" * 80)
    print(f"   ✅ Se consultaron {len(documentos_unicos)} documentos únicos")
    print(f"   📅 Discursos desde {min(fechas_unicas) if fechas_unicas else 'N/A'}")
    print(f"   🔍 {len(documentos)} fragmentos analizados para la respuesta")
    print(f"   📊 Cobertura temporal: {len(fechas_unicas)} fechas diferentes")

    return resultado

# Función adicional para mostrar solo un resumen rápido de fuentes
def consulta_rapida(pregunta, mostrar_fuentes_detalladas=False):
    """
    Versión rápida para consultas simples
    """
    print(f"\n⚡ CONSULTA RÁPIDA")
    print(f"Pregunta: {pregunta}")
    print("\n" + "="*60)

    resultado = sistema_rag_mejorado({"query": pregunta})

    print(f"🤖 RESPUESTA:")
    print(resultado["result"])

    if mostrar_fuentes_detalladas:
        documentos = resultado['source_documents']
        print(f"\n📚 FUENTES ({len(documentos)} fragmentos):")
        for i, doc in enumerate(documentos, 1):
            titulo = doc.metadata.get('titulo', 'Título no disponible')
            fecha = doc.metadata.get('fecha_publicacion', 'N/A')
            contenido = doc.page_content.replace('\n', ' ')[:100] + "..."
            print(f"   {i}. [{fecha}] {titulo}")
            print(f"      📄 {contenido}")
    else:
        documentos_unicos = list(set([doc.metadata.get('titulo', 'Título no disponible') for doc in resultado['source_documents']]))
        print(f"\n📁 Documentos consultados: {len(documentos_unicos)}")
        for i, titulo in enumerate(documentos_unicos, 1):
            print(f"   {i}. {titulo}")

    return resultado

# Probemos con preguntas que necesitan múltiples fuentes
print("🚀 PROBANDO CONSULTA COMPLEJA MEJORADA")
consulta_compleja("¿Qué dice Javier Milei sobre la economía y las reformas estructurales?")


🚀 PROBANDO CONSULTA COMPLEJA MEJORADA

🧩 CONSULTA COMPLEJA
📋 Pregunta: ¿Qué dice Javier Milei sobre la economía y las reformas estructurales?

🤖 RESPUESTA INTEGRAL:
¡Hola! Con gusto te proporciono la información sobre la economía y las reformas estructurales según los discursos disponibles del Presidente Javier Milei.

El Presidente Javier Milei enfatiza que el ordenamiento de la macroeconomía es la base fundamental para el crecimiento económico y que las reformas estructurales son urgentes para dinamizar la microeconomía. Destaca la importancia de la secuencia correcta en la implementación de estas reformas, especialmente en lo que respecta a la apertura comercial, y subraya el equilibrio fiscal como un pilar esencial para el crecimiento sostenido.

Aquí te detallo los puntos específicos:

1.  **Ordenamiento Macroeconómico como Base del Crecimiento:**
    *   Durante el último año y medio, se dedicaron a "ordenar la macro a toda costa", considerándola la base del crecimiento económico

{'query': '¿Qué dice Javier Milei sobre la economía y las reformas estructurales?',
 'result': '¡Hola! Con gusto te proporciono la información sobre la economía y las reformas estructurales según los discursos disponibles del Presidente Javier Milei.\n\nEl Presidente Javier Milei enfatiza que el ordenamiento de la macroeconomía es la base fundamental para el crecimiento económico y que las reformas estructurales son urgentes para dinamizar la microeconomía. Destaca la importancia de la secuencia correcta en la implementación de estas reformas, especialmente en lo que respecta a la apertura comercial, y subraya el equilibrio fiscal como un pilar esencial para el crecimiento sostenido.\n\nAquí te detallo los puntos específicos:\n\n1.  **Ordenamiento Macroeconómico como Base del Crecimiento:**\n    *   Durante el último año y medio, se dedicaron a "ordenar la macro a toda costa", considerándola la base del crecimiento económico.\n    *   *Discurso del Presidente de la Nación, Javier Milei

In [ ]:
consulta_compleja("De que trato el discurso de asuncion de Milei?")


🧩 CONSULTA COMPLEJA
📋 Pregunta: De que trato el discurso de asuncion de Milei?

🤖 RESPUESTA INTEGRAL:
El discurso de asunción del Presidente Javier Milei trató sobre el inicio de una nueva era para Argentina, marcando el fin de un período de decadencia y el comienzo de la reconstrucción del país. Enfatizó la voluntad de cambio expresada por los argentinos y delineó los principios de su gobierno.

Aquí los detalles específicos:

1.  **Nueva Era y Fin de la Decadencia:** Milei declaró que "Hoy comienza una nueva era en Argentina", dando por terminada "una larga historia de decadencia y declive" y comenzando "el camino de la reconstrucción de nuestro país". Afirmó que los argentinos expresaron una "voluntad de cambio que ya no tiene retorno", enterrando "décadas de fracaso, peleas intestinas y disputas sin sentido" que llevaron al país a la ruina.
    *   **Discurso:** DISCURSO DEL PRESIDENTE JAVIER MILEI EN SU ASUNCIÓN / Discurso de Javier Milei en acto de asunción.
    *   **Fecha:** N

{'query': 'De que trato el discurso de asuncion de Milei?',
 'result': 'El discurso de asunción del Presidente Javier Milei trató sobre el inicio de una nueva era para Argentina, marcando el fin de un período de decadencia y el comienzo de la reconstrucción del país. Enfatizó la voluntad de cambio expresada por los argentinos y delineó los principios de su gobierno.\n\nAquí los detalles específicos:\n\n1.  **Nueva Era y Fin de la Decadencia:** Milei declaró que "Hoy comienza una nueva era en Argentina", dando por terminada "una larga historia de decadencia y declive" y comenzando "el camino de la reconstrucción de nuestro país". Afirmó que los argentinos expresaron una "voluntad de cambio que ya no tiene retorno", enterrando "décadas de fracaso, peleas intestinas y disputas sin sentido" que llevaron al país a la ruina.\n    *   **Discurso:** DISCURSO DEL PRESIDENTE JAVIER MILEI EN SU ASUNCIÓN / Discurso de Javier Milei en acto de asunción.\n    *   **Fecha:** No se menciona una fecha e

## SISTEMA DE CARGA DE ARCHIVOS PARA JUPYTER



In [ ]:
!pip install pypdf -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 8.2 MB/s eta 0:00:00


In [ ]:
# @title
import os
import glob
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from langchain.document_loaders import PyPDFLoader, TextLoader, UnstructuredWordDocumentLoader
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Configurar el text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ".", " "]
)

# Widgets para interfaz interactiva
progress = widgets.FloatProgress(value=0.0, min=0, max=1.0, description='Progreso:')
status_text = widgets.HTML(value="<i>Listo para cargar archivos...</i>")
file_stats = widgets.HTML(value="")
result_display = widgets.Output()

def detectar_o_crear_carpeta(ruta_carpeta="./documentos/"):
    """
    Detecta si la carpeta existe, si no, la crea y da instrucciones
    """
    if os.path.exists(ruta_carpeta):
        archivos = glob.glob(os.path.join(ruta_carpeta, "*"))
        archivos_soportados = [f for f in archivos if any(f.endswith(ext) for ext in ['.pdf', '.txt', '.docx'])]

        with result_display:
            if archivos_soportados:
                print(f"✅ Carpeta encontrada: {ruta_carpeta}")
                print(f"📁 Archivos detectados: {len(archivos_soportados)}")
                for archivo in archivos_soportados[:5]:  # Mostrar primeros 5
                    print(f"   📄 {os.path.basename(archivo)}")
                if len(archivos_soportados) > 5:
                    print(f"   ... y {len(archivos_soportados) - 5} más")
            else:
                print(f"📁 Carpeta encontrada pero sin archivos PDF/TXT/DOCX")
                print("💡 Agrega archivos a la carpeta 'documentos/' y vuelve a intentar")

        return True, archivos_soportados
    else:
        # Crear la carpeta
        os.makedirs(ruta_carpeta, exist_ok=True)

        with result_display:
            print(f"📂 Se creó la carpeta: {ruta_carpeta}")
            print("\n📋 INSTRUCCIONES PARA AGREGAR ARCHIVOS:")
            print("1. Ve a la pestaña 'Archivos' en el lateral izquierdo")
            print("2. Busca la carpeta 'documentos'")
            print("3. Arrastra tus archivos PDF, TXT o DOCX allí")
            print("4. Vuelve a hacer clic en 'Cargar carpeta documentos/'")
            print("\n💡 También puedes usar 'Cargar archivos específicos' para rutas exactas")

        return False, []

def cargar_archivos_jupyter(archivos_lista):
    """
    Versión optimizada para Jupyter - carga archivos con barra de progreso
    """
    documentos = []
    total_archivos = len(archivos_lista)

    if total_archivos == 0:
        status_text.value = "<span style='color: orange'>⚠️ No hay archivos para procesar</span>"
        return []

    status_text.value = f"<span style='color: blue'>📁 Procesando {total_archivos} archivos...</span>"

    for i, archivo_path in enumerate(archivos_lista):
        try:
            # Actualizar progreso
            progress.value = (i + 1) / total_archivos
            nombre_archivo = os.path.basename(archivo_path)

            # Mostrar archivo actual
            status_text.value = f"<span style='color: blue'>📖 Leyendo: {nombre_archivo} ({i+1}/{total_archivos})</span>"

            if archivo_path.lower().endswith('.pdf'):
                loader = PyPDFLoader(archivo_path)
                docs = loader.load()
                tipo = "📄 PDF"

            elif archivo_path.lower().endswith('.txt'):
                loader = TextLoader(archivo_path, encoding='utf-8')
                docs = loader.load()
                tipo = "📝 TXT"

            elif archivo_path.lower().endswith('.docx'):
                loader = UnstructuredWordDocumentLoader(archivo_path)
                docs = loader.load()
                tipo = "📋 DOCX"

            else:
                with result_display:
                    print(f"❌ Formato no soportado: {nombre_archivo}")
                continue

            # Agregar metadata
            for doc in docs:
                doc.metadata.update({
                    "fuente": "archivo_local",
                    "nombre_archivo": nombre_archivo,
                    "ruta_archivo": archivo_path,
                    "tipo_archivo": tipo.strip()
                })

            documentos.extend(docs)

            # Mostrar éxito en el output
            with result_display:
                print(f"✅ {tipo}: {nombre_archivo} → {len(docs)} páginas/secciones")

        except Exception as e:
            with result_display:
                print(f"❌ Error en {archivo_path}: {str(e)}")
            continue

    progress.value = 1.0
    return documentos

def procesar_archivos_interactivo(vectorstore, archivos_lista):
    """
    Función interactiva para procesar archivos en Jupyter
    """
    # Limpiar output anterior
    result_display.clear_output()

    with result_display:
        print("🔄 INICIANDO PROCESAMIENTO DE ARCHIVOS")
        print("=" * 50)

    # 1. Cargar archivos
    documentos = cargar_archivos_jupyter(archivos_lista)

    if not documentos:
        status_text.value = "<span style='color: red'>❌ No se pudieron cargar archivos</span>"
        return 0, 0

    # 2. Dividir en fragmentos
    with result_display:
        print(f"\n🔪 Dividiendo {len(documentos)} documentos en fragmentos...")

    fragmentos = text_splitter.split_documents(documentos)

    with result_display:
        print(f"📝 Fragmentos creados: {len(fragmentos)}")

    # 3. Agregar al vectorstore
    with result_display:
        print(f"\n📚 Agregando al vectorstore...")

    try:
        vectorstore.add_documents(fragmentos)

        with result_display:
            print(f"✅ ✅ {len(fragmentos)} fragmentos agregados exitosamente!")

        # 4. Mostrar estadísticas bonitas
        mostrar_estadisticas(documentos, fragmentos)

        status_text.value = "<span style='color: green'>🎉 ¡Carga completada exitosamente!</span>"

    except Exception as e:
        with result_display:
            print(f"❌ Error agregando documentos: {e}")
        status_text.value = "<span style='color: red'>❌ Error en la carga</span>"
        return len(documentos), 0

    return len(documentos), len(fragmentos)

def mostrar_estadisticas(documentos, fragmentos):
    """
    Muestra estadísticas formateadas para Jupyter
    """
    # Contar tipos de archivos
    tipos_archivos = {}
    for doc in documentos:
        archivo = doc.metadata.get('nombre_archivo', '')
        if archivo.endswith('.pdf'):
            tipos_archivos['PDF'] = tipos_archivos.get('PDF', 0) + 1
        elif archivo.endswith('.txt'):
            tipos_archivos['TXT'] = tipos_archivos.get('TXT', 0) + 1
        elif archivo.endswith('.docx'):
            tipos_archivos['DOCX'] = tipos_archivos.get('DOCX', 0) + 1

    with result_display:
        print(f"\n📊 RESUMEN DE CARGA:")
        print("=" * 50)
        print(f"   📄 Documentos originales: {len(documentos)}")
        print(f"   🧩 Fragmentos generados: {len(fragmentos)}")
        print(f"   📈 Ratio fragmentos/doc: {len(fragmentos)/len(documentos):.1f}")

        if tipos_archivos:
            print(f"\n   📁 Tipos de archivos:")
            for tipo, cantidad in tipos_archivos.items():
                print(f"      • {tipo}: {cantidad}")

        print(f"\n   💾 Base de conocimiento actualizada")
        print("=" * 50)

# FUNCIONES DE CONVENIENCIA MEJORADAS
def cargar_desde_carpeta(vectorstore, ruta_carpeta="./documentos/"):
    """
    Carga todos los archivos de una carpeta - CON DETECCIÓN AUTOMÁTICA
    """
    # Detectar o crear carpeta
    carpeta_existe, archivos_lista = detectar_o_crear_carpeta(ruta_carpeta)

    if not carpeta_existe or not archivos_lista:
        return 0, 0

    archivos_lista.sort()  # Ordenar alfabéticamente

    with result_display:
        print(f"📂 Carpeta: {ruta_carpeta}")
        print(f"🔍 Procesando {len(archivos_lista)} archivos...")

    return procesar_archivos_interactivo(vectorstore, archivos_lista)

def cargar_archivos_especificos(vectorstore, lista_rutas):
    """
    Carga una lista específica de archivos - CON VERIFICACIÓN MEJORADA
    """
    # Verificar que los archivos existen
    archivos_validos = []
    archivos_no_encontrados = []

    for ruta in lista_rutas:
        if os.path.exists(ruta):
            archivos_validos.append(ruta)
        else:
            archivos_no_encontrados.append(ruta)

    if archivos_no_encontrados:
        with result_display:
            print("⚠️ Archivos no encontrados:")
            for ruta in archivos_no_encontrados:
                print(f"   ❌ {ruta}")

    if not archivos_validos:
        with result_display:
            print("❌ No hay archivos válidos para cargar")
        return 0, 0

    with result_display:
        print(f"🔍 Procesando {len(archivos_validos)} archivos específicos...")

    return procesar_archivos_interactivo(vectorstore, archivos_validos)

# WIDGETS PARA INTERFAZ VISUAL MEJORADA
def crear_interfaz_carga():
    """
    Crea una interfaz visual mejorada para cargar archivos
    """
    # Botones de acción
    btn_carpeta = widgets.Button(
        description="📂 Cargar carpeta 'documentos/'",
        button_style='primary',
        tooltip='Carga todos los archivos de la carpeta documentos/',
        layout=widgets.Layout(width='300px', height='40px')
    )

    btn_explorar = widgets.Button(
        description="🔍 Explorar carpeta actual",
        button_style='info',
        tooltip='Ver qué archivos hay disponibles',
        layout=widgets.Layout(width='200px', height='40px')
    )

    btn_manual = widgets.Button(
        description="🔧 Cargar archivos específicos",
        button_style='info',
        tooltip='Especificar rutas manualmente',
        layout=widgets.Layout(width='250px', height='40px')
    )

    btn_limpiar = widgets.Button(
        description="🧹 Limpiar resultados",
        button_style='warning',
        layout=widgets.Layout(width='150px', height='40px')
    )

    # Campo para rutas manuales
    texto_rutas = widgets.Textarea(
        value="",
        placeholder="Pega las rutas de archivos, una por línea:\n/documentos/doc1.pdf\n/documentos/doc2.txt\nO usa rutas absolutas: /content/mi_archivo.pdf",
        description="Archivos:",
        disabled=False,
        layout=widgets.Layout(width="90%", height="100px")
    )

    # Función para botón de carpeta
    def on_btn_carpeta_click(b):
        cargar_desde_carpeta(vectorstore)

    # Función para explorar
    def on_btn_explorar_click(b):
        result_display.clear_output()
        detectar_o_crear_carpeta("./documentos/")

    # Función para botón manual
    def on_btn_manual_click(b):
        if texto_rutas.value.strip():
            rutas = [r.strip() for r in texto_rutas.value.split('\n') if r.strip()]
            cargar_archivos_especificos(vectorstore, rutas)
        else:
            with result_display:
                print("⚠️ Ingresa al menos una ruta de archivo")

    # Función para limpiar
    def on_btn_limpiar_click(b):
        result_display.clear_output()
        status_text.value = "<i>Resultados limpiados...</i>"
        progress.value = 0.0

    # Conectar botones
    btn_carpeta.on_click(on_btn_carpeta_click)
    btn_explorar.on_click(on_btn_explorar_click)
    btn_manual.on_click(on_btn_manual_click)
    btn_limpiar.on_click(on_btn_limpiar_click)

    # Diseño de la interfaz
    controles = widgets.VBox([
        widgets.HTML(value="<h3>🎯 Opciones de Carga:</h3>"),
        widgets.HBox([btn_carpeta, btn_explorar]),
        widgets.HBox([btn_manual, btn_limpiar]),
        widgets.HTML(value="<hr>"),
        widgets.HTML(value="<h4>📋 Carga Manual de Archivos:</h4>"),
        texto_rutas
    ])

    return widgets.VBox([
        widgets.HTML(value="<h2>🗂️ Cargador de Archivos para RAG</h2>"),
        widgets.HTML(value="<p>Carga archivos PDF, TXT y DOCX a tu sistema de consultas</p>"),
        controles,
        widgets.HTML(value="<hr>"),
        progress,
        status_text,
        file_stats,
        widgets.HTML(value="<h4>📝 Log de ejecución:</h4>"),
        result_display
    ])

# INICIALIZAR INTERFAZ
def inicializar_cargador():
    """
    Inicializa y muestra el cargador interactivo
    """
    display(crear_interfaz_carga())

    # Mostrar estado inicial
    with result_display:
        print("🔍 Estado inicial del sistema:")
    detectar_o_crear_carpeta("./documentos/")

# USO RÁPIDO (sin interfaz)
def carga_rapida(vectorstore, ruta_archivo):
    """
    Función rápida para cargar un solo archivo
    """
    if os.path.exists(ruta_archivo):
        return procesar_archivos_interactivo(vectorstore, [ruta_archivo])
    else:
        print(f"❌ Archivo no encontrado: {ruta_archivo}")
        return 0, 0

# FUNCIÓN PARA SUBIR ARCHIVOS DIRECTAMENTE EN COLAB
def instrucciones_subida_colab():
    """
    Muestra instrucciones específicas para Google Colab
    """
    with result_display:
        print("🚀 INSTRUCCIONES PARA GOOGLE COLAB:")
        print("=" * 50)
        print("1. Haz clic en el icono de 📁 (Archivos) en el lateral izquierdo")
        print("2. Haz clic derecho en la carpeta 'documentos' o en espacio vacío")
        print("3. Selecciona 'Subir' y elige tus archivos PDF/TXT/DOCX")
        print("4. Espera a que se completen las subidas")
        print("5. Vuelve a esta celda y haz clic en 'Cargar carpeta documentos/'")
        print("\n💡 También puedes usar:")
        print("   • 'Explorar carpeta actual' para ver qué hay")
        print("   • 'Cargar archivos específicos' para rutas exactas")

# EJECUTAR ESTO EN JUPYTER PARA USAR LA INTERFAZ:
print("✅ Sistema de carga listo - Ejecuta: inicializar_cargador()")

✅ Sistema de carga listo - Ejecuta: inicializar_cargador()


In [ ]:
inicializar_cargador()